#### Analyzing data from the reader study; creating corresponding plots, and calculating metrics.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sst
import csv

from ImportantFunctions import getLabelMeanPerAngle, getLabelsFromDf, getLabelsFromDf_readerLevel 

#### 1. Read and store data

In [ ]:
df = pd.read_csv("ResultsReaderStudy_short.csv")

quality, confidence, artifact, dice = getLabelsFromDf(df)

quality_sparse, quality_unet = quality[0], quality[1]
confidence_sparse, confidence_unet = confidence[0], confidence[1]
artifact_sparse, artifact_unet = artifact[0], artifact[1]
dice_sparse, dice_unet = dice[0], dice[1]

#### 2. Clustered wilcoxon signed rank test

In [ ]:
# create clusters for three readers:
clusters = np.ones((np.shape(quality_sparse[0]))) #reader 1
clusters[19:2*19] = clusters[19:2*19]*2 # reader 2
clusters[2*19:3*19] = clusters[2*19:3*19]*3 # reader 3

# print p-values
print('quality')
for i in range(5):
    print(2**(i+4),' views')
    z, p = clustered_signed_rank_wilcoxon(quality_sparse[i], quality_unet[i], clusters)
    print('p-value: ',round(p,4))
print('\n confidence')   
for i in range(5):
    print(2**(i+4),' views')
    z, p = clustered_signed_rank_wilcoxon(confidence_sparse[i], confidence_unet[i], clusters)
    print('p-value: ',round(p,5))
print('\n artifact')
for i in range(5):
    print(2**(i+4),' views')
    z, p = clustered_signed_rank_wilcoxon(artifact_sparse[i], artifact_unet[i], clusters)
    print('p-value: ',round(p,4))

print('\n DSC')
for i in range(1,5):
    print(2**(i+4),' views')
    if len(dice_sparse[i])-len(dice_unet[i])!=0 and len(dice_sparse_s[i])> len(dice_unet_s[i]):
        _, p = clustered_signed_rank_wilcoxon(dice_sparse[i][:len(dice_unet[i])], dice_unet[i], clusters)
    elif len(dice_sparse[i])==len(dice_unet[i]):
        _, p = clustered_signed_rank_wilcoxon(dice_sparse[i], dice_unet[i], clusters)
    else:
        _, p = clustered_signed_rank_wilcoxon(dice_sparse[i], dice_unet[i][:len(dice_sparse[i])], clusters)
    print(round(p,4))

#### 3. Plot results

In [ ]:
width = 0.4
x = np.arange(5)

fig, axs = plt.subplots(2, 2, figsize=(15, 15))

# Plot each subplot
plot_subplot(axs[0, 0], quality_sparse_mean, quality_unet_mean, 'A) Quality Labels', 'Mean (1-6 possible)', ylim=5.05)
plot_subplot(axs[0, 1], confidence_sparse_mean, confidence_unet_mean, 'B) Confidence Labels', 'Mean (1-6 possible)', ylim=5.05)
plot_subplot(axs[1, 0], artifact_sparse_mean, artifact_unet_mean, 'C) Artifact Labels', 'Mean (1-4 possible)', ylim=5.05)
plot_subplot(axs[1, 1], dice_sparse_mean_s, dice_unet_mean_s, 'D) Segmentation Overlap', 'Mean Dice Similarity Coefficient')

handles, labels = axs[0, 0].get_legend_handles_labels()
fig.legend(handles, labels, loc="upper right", bbox_to_anchor=(1.03, 0.95), fontsize="x-large")
plt.tight_layout()
#plt.savefig("resullts.pdf", format='pdf')
plt.show()

#### 4. Get confusion matrix values

In [ ]:
# a) define dictionaty
VFT_tpo = {} #values for False and True cases 
keys = [16, 32, 64, 128, 256]
metrics = {"tp": 0, "tn": 0, "fp": 0, "fn": 0}

for category in ["predct", "sparse"]:
    VFT_tpo[category] = {key: dict(metrics) for key in keys}

reader = ['r1','r2','r3']
for idx in range(len(df["patientid"])):
    method=df["method"][idx]
    angle=df["angles"][idx]
    sum_gtmask=df["len_gtmask"][idx]
    sum_rmask=df["len_rmask"][idx]
    dice=df["dice"][idx]
    if df["reader"][idx] in reader:
        if sum_gtmask==0 and sum_rmask==0:
            VFT_tpo[method][angle]["tn"]+=1
            #print("tn case", dice, df["reader"][idx], df["patientid"][idx], df["angles"][idx], df["method"][idx])
        elif sum_gtmask==0 and sum_rmask>0:
            VFT_tpo[method][angle]["fp"]+=1
            #print("XXXXX fp case", sum_rmask, df["reader"][idx], df["patientid"][idx], df["angles"][idx], df["method"][idx])
        elif sum_gtmask>0 and sum_rmask==0:
            VFT_tpo[method][angle]["fn"]+=1
            #print("fn no tumor found case", dice, df["reader"][idx], df["patientid"][idx], df["angles"][idx], df["method"][idx])
        elif sum_gtmask>0 and sum_rmask>0 and dice>0:
            VFT_tpo[method][angle]["tp"]+=1
            #print("tp case", dice, df["reader"][idx], df["patientid"][idx], df["angles"][idx], df["method"][idx])
        elif sum_gtmask>0 and sum_rmask>0 and dice==0:
            VFT_tpo[method][angle]["fn"]+=1
            #print("fn met at wrong location case", dice, df["reader"][idx], df["patientid"][idx], df["angles"][idx], df["method"][idx])
            #print(df["reader"][idx], df["patientid"][idx], df["angles"][idx], df["method"][idx], "marked tumor at wrong location")

In [ ]:
# b) print confusion matrix (and uncomment to save to csv file)

#f = open('R?_CM.csv', 'w')
#writer = csv.writer(f)
for method in VFT_tpo.keys():
    #writer.writerow([method, "", "", ""])
    print("method: %s"%method)
    for angle in VFT_tpo[method].keys():
        print("    %s projection angles"%angle)
        print("        reader/true    +        -")
        print("                +      %s       %s           sum:%s"%(VFT_tpo[method][angle]["tp"], VFT_tpo[method][angle]["fp"], VFT_tpo[method][angle]["tp"]+VFT_tpo[method][angle]["fp"]))
        print("                -      %s       %s           sum:%s"%(VFT_tpo[method][angle]["fn"], VFT_tpo[method][angle]["tn"], VFT_tpo[method][angle]["fn"]+VFT_tpo[method][angle]["tn"]))
        print("                                                     ")
        print("                  sum: %s       %s"%(VFT_tpo[method][angle]["tp"]+VFT_tpo[method][angle]["fn"], VFT_tpo[method][angle]["fp"]+VFT_tpo[method][angle]["tn"]))
        print("                                                     ")
        
        #writer.writerow([angle, "projection angles", "", ""])
        #writer.writerow(["r_t", "+", "-", "sum"])
        #writer.writerow(["+", VFT_tpo[method][angle]["tp"], VFT_tpo[method][angle]["fp"], VFT_tpo[method][angle]["tp"]+VFT_tpo[method][angle]["fp"]])
        #writer.writerow(["-", VFT_tpo[method][angle]["fn"], VFT_tpo[method][angle]["tn"], VFT_tpo[method][angle]["fn"]+VFT_tpo[method][angle]["tn"]])
        #writer.writerow(["sum", VFT_tpo[method][angle]["tp"]+VFT_tpo[method][angle]["fn"], VFT_tpo[method][angle]["fp"]+VFT_tpo[method][angle]["tn"], ""])
#f.close()

In [ ]:
# c) print corresponding sensitivity, specificity, F1, and negative predictive value metrics (and uncomment to save to csv file)

#f = open('R?_Metrics.csv', 'w')
#writer = csv.writer(f)
for method in VFT_tpo.keys():
    #writer.writerow([method, "", "", ""])
    print("method: %s"%method)
    for angle in VFT_tpo[method].keys():
        Se = VFT_tpo[method][angle]["tp"] / (VFT_tpo[method][angle]["tp"]+VFT_tpo[method][angle]["fn"])
        Sp = VFT_tpo[method][angle]["tn"] / (VFT_tpo[method][angle]["fp"]+VFT_tpo[method][angle]["tn"])
        F1 = (2*VFT_tpo[method][angle]["tp"])/((2*VFT_tpo[method][angle]["tp"]) + VFT_tpo[method][angle]["fp"] + VFT_tpo[method][angle]["fn"])
        NPV = VFT_tpo[method][angle]["tn"] / (VFT_tpo[method][angle]["tn"] + VFT_tpo[method][angle]["fn"])
        
        print("    %s projection angles"%angle)
        print("        Se:     %s"%(round(Se, 2)))
        print("        Sp:     %s"%(round(Sp, 2)))
        print("        F1:     %s"%(round(F1, 2)))
        print("        NPV:    %s"%(round(NPV, 2)))
        print("                ")
        
        #writer.writerow([angle, "projection angles", "", ""])
        #writer.writerow(["Se", Se])
        #writer.writerow(["Sp", Sp])
        #writer.writerow(["F1", F1])
        #writer.writerow(["NPV", NPV])
#f.close()